# Titanic

In this example, we use the Titanic dataset to predict the survival of passengers on the Titanic. You can download the Jupyter Notebook of the study <a href="titanic.ipynb">here</a>.

## Initialization


In [ ]:
from verticapy import *
new_auto_connection({"host": "34.237.154.116", 
                     "port": "5433", 
                     "database": "testdrive", 
                     "password": "password", 
                     "user": "dbadmin"},
                    name = "VerticaDSN")
# Set the primary auto-connection
change_auto_connection("VerticaDSN")

Use the following command to allow Matplotlib to display graphics.

In [ ]:
%matplotlib inline

Let's load the Titanic dataset.

In [ ]:
from verticapy.datasets import load_titanic
from verticapy import *
titanic = load_titanic()
display(titanic)

In [ ]:
titanic = vDataFrame('titanic')
titanic

## Data Exploration and Preparation

Let's explore the data by displaying descriptive statistics of all the columns.

In [ ]:
titanic.describe(method = "categorical")

The columns "body" (passenger ID), "home.dest" (passenger origin/destination), "embarked" (origin port) and "ticket" (ticket ID) shouldn't influence survival, so we can ignore these.

Let's focus our analysis on the columns "name" and "cabin." We'll begin with the passgeners' names.

In [ ]:
%load_ext verticapy.sql

In [ ]:
%%sql
select * from models;

In [ ]:
from verticapy.learn.preprocessing import CountVectorizer
model = CountVectorizer("name_voc4")
model.drop()
model.fit(titanic, ["Name"]).transform()

In [ ]:
%load_ext verticapy.sql

In [ ]:
%%sql
SELECT
    REGEXP_SUBSTR(name, '([A-Za-z]+\.)') as TITLE,
    count(*),
    AVG(survived)
FROM titanic GROUP BY 1 order by 3 desc;    

We can extract each passenger's title through their name, which might come in handy.

Let's move on to the cabins.

In [ ]:
from verticapy.learn.preprocessing import CountVectorizer
model = CountVectorizer("cabin_voc9")
model.drop()
model.fit("titanic", ["cabin"]).transform()

Here, we have the cabin IDs, the letter of which represents a certain position on the boat. Let's look at the number of occurences.

In [ ]:
CountVectorizer("cabin_voc10").fit("titanic", ["cabin"]).transform(
                )["token"].str_slice(1, 1).groupby(
                columns = ["token"], expr = ["SUM(cnt)"]).head(30)

NULL values for "cabin" might represent passengers without a cabin. If this is the case, then these are missing values not at random (MNAR).

NULL values for "boat" represent passengers who have a dedicated "lifeboat." We can drop the useless columns and encode the others.

In [ ]:
titanic.drop(["body", "home.dest", "embarked", "ticket"])

In [ ]:
titanic["cabin"].str_slice(1, 1)["name"].str_extract(
        ' ([A-Za-z]+)\.')["boat"].fillna(
        method = "0ifnull")["cabin"].fillna("No Cabin")

Our assumption about the meaning of a NULL value of "cabin" turned out to be incorrect; after all, first class passengers should have a cabin. This means that the "cabin" column has far too many missing values at random (MAR). We'll have to drop it.

In [ ]:
titanic["cabin"].drop()

Let's look at descriptive statistics of the entire Virtual Dataframe.

In [ ]:
titanic.describe(method = "all")

This method will help us better understand our data. Let's draw histogram for "age".

In [ ]:
titanic["age"].hist()

We can also perform a Jarque-Bera test to test our hypothesis.

In [ ]:
from verticapy.stats import jarque_bera
# Does this follow a normal distribution?
jarque_bera(titanic, "age", alpha = 0.01)

Notice also that the column "fare" has many outliers (The maximum of 512.33 is much greater than the 9th decile of 79.13). Most of the passengers traveled in 3rd class (median of pclass = 3) and much more...

The "sibsp" column represents the number of siblings, while the "parch" column represents the number of parents and children. We can use these to create a new feature: "family_size".

In [ ]:
titanic["family_size"] = titanic["parch"] + titanic["sibsp"] + 1

Let's deal with the outliers. We have several options to find them (LocalOutlier Factor, DBSCAN, k-means...) but we will just use winsorization. Our target is the "fare" distribution since it has so many outliers. 

In [ ]:
# Limit extreme values, all data above 97th percentile, is set to 97th percentile
titanic["fare"].fill_outliers(method = "winsorize", 
                              alpha = 0.03)

Let's encode the column "sex" so we can use it with numerical methods. 

In [ ]:
titanic["sex"].label_encode()

In [ ]:
print(titanic.current_relation())

The column "age" has too many missing values and since most machine learning algorithms don't handle missing values, we need to use imputation techniques. Let's fill the missing values using the average "age" of the passengers that have the same "pclass" and "sex."

In [ ]:
titanic["age"].fillna(method = "mean", by = ["pclass", "sex"])

Let's draw the correlation matrix to see the links between variables.

In [ ]:
titanic.corr(method = "spearman")

Fare correlates strongly with family size. This is about what you would expect: it makes sense that the larger the family, the more tickets they'd have to buy, and the greater the fare.

Survival correlates strongly with whether or not a passenger has a lifeboat (the "boat" variable). Still, to increase the generality of our model, we should avoid predictions based on just one variable. Let's split the study into two use cases:

<ul>
    <li>Passengers with a lifeboat</li>
    <li>Passengers without a lifeboat</li>
</ul>

Before we move on: we did a lot of work to clean up this data, but we haven't saved anything to our Vertica database! We can look at our vDataFrame to be sure.

In [ ]:
print(titanic.current_relation())

Let see what's happening when we aggregate and turn on SQL generation.

In [ ]:
set_option("sql_on", True)
titanic.avg()

VerticaPy dynamically generates SQL code whenever you make modifications to your data. It will also store computed aggregations to avoid unnecessary recomputation. If we filter anything, it will update the catalog with our modifications.

In [ ]:
set_option("sql_on", False)
print(titanic.info())

Let's move on to data modeling. Save the vDataframe in your Vertica database.

In [ ]:
from verticapy import drop_view
drop_view("titanic_boat")
drop_view("titanic_no_boat")

titanic_boat = titanic.search(conditions = ["boat = 1"])
titanic_no_boat = titanic.search(conditions = ["boat = 0"])
#titanic.to_dbtitanic.filter(titanic["boat"] == 1)

In [ ]:
titanic_boat.to_db(name = '"public"."titanic_boat"')

In [ ]:
titanic_no_boat.to_db(name = '"public"."titanic_no_boat"')

## Machine Learning

### Passengers with a lifeboat


First, let's look at the number of survivors.

In [ ]:
from verticapy import vDataFrame
titanic_boat = vDataFrame("titanic_boat")
titanic_boat["survived"].describe()

We have 9 deaths. Let's try to understand why these passengers died.

In [ ]:
#titanic_boat.filter(titanic_boat["survived"] == 0).head(10)
titanic_boat.search(conditions = ["survived = 0"])

Apart from third-class passengers, it doesn't seem like these passengers have any predictors for their deaths. Making a model out of this would be unhelpful. 

### Passengers without a lifeboat

Let's move on to passengers without a lifeboat. 

In [ ]:
titanic_boat = vDataFrame("titanic_no_boat")
titanic_boat["survived"].describe()

Only 20 survived. Let's find out why.

In [ ]:
#titanic_boat.filter(titanic_boat["survived"] == 1).head(20)
titanic_boat.search(conditions = ["survived = 1"])

Most survivors seem to be women. Let's build a model with this.

One of our predictors is categorical: the passenger title. Some of these predictors are corrleated, so it'd be best to work with a non-linear classifier that can handle that. In this case, random forest seems to be perfect. Let's evaluate it with a cross validation.

In [ ]:
from verticapy.learn.ensemble import RandomForestClassifier
from verticapy.learn.model_selection import cross_validate

predictors = titanic.get_columns(exclude_columns = ['"survived"'])
response = "survived"
relation = "titanic_no_boat"
model = RandomForestClassifier("rf_titanic", 
                               n_estimators = 40, 
                               max_depth = 4)
cross_validate(model, relation, predictors, response)

This dataset is pretty unbalanced so we'll use AUC to evaluate it.

The model is very good with an average greater than 0.9!

We can now build a model with the entire dataset.

In [ ]:
model.fit(relation, predictors, response)

Let's look at the features importance.

In [ ]:
model.features_importance()

As expected, a passenger's title and the sex are the most important predictors of survival.

## Conclusion

We've solved our problem in a Pandas-like way, all without ever loading data into memory!